<a href="https://colab.research.google.com/github/analyticsariel/projects/blob/master/040_Get_Rental_Comparables_RealtyMole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Steps
### Get Keys
1) Get Free [Rapid API Key by creating an account](https://rapidapi.com/marketplace)

Store each key in a central and private location.

### Suscribe to APIs

1) Suscribe to [Realty Mole on Rapid API](https://rapidapi.com/realtymole/api/realty-mole-property-api/endpoints)

## <font color="blue">Imports</font>

In [ ]:
# install packages
# pip
"""
pip is used to download and install packages directly from PyPI. 
PyPI is a specialized package manager that only deals with python packages. 
"""

!pip install gmaps -q
!pip install ipywidgets -q
!pip install widgetsnbextension -q

     |████████████████████████████████| 1.1 MB 15.3 MB/s 


## <font color="blue">Install Packages</font>

In [ ]:
# working with files
from google.colab import drive, files
# api post/get
import requests
from bs4 import BeautifulSoup
# data manipulation
import json
import pandas as pd
import numpy as np
# plotting
import gmaps
import ipywidgets as widgets
from ipywidgets.embed import embed_minimal_html
import IPython

## <font color="blue">Functions</font>

### Realty Mole

In [ ]:
def realty_mole_rent_estimate(rapid_api_key,
                           street,
                           city,
                           state,
                           squareFootage=None,
                           bathrooms=None,
                           bedrooms=None,
                           propertyType=None,
                           daysOld=None,
                           compCount=5):
    """
    Based on the provided property details this API will return 
    a rent estimate along with comparable rental properties. 
    Most of the response values are self-explanatory. 
    Please note that distance measurements are given in kilometers.
    
    The Find Property API Web Service is located at: 
    https://rapidapi.com/realtymole/api/realty-mole-property-api/endpoints

    Parameters
    ----------
    @rapid_api_key [string]: Key to access data from Rapid API
    @address [string]: The address of the property to search
    @city [string]: The city of the property to search
    @state [string]: The state of the property to search
    @squareFootage [string]: The square footage of the property to search
    @bathrooms [string]: The bathrooms of the property to search
    @comp_count [string]: The number of comparable properties to evaluate sales price

    Returns
    -------
    [json] API response

    """
    # get address
    address = "{0}, {1}, {2}".format(str(street), 
                                     str(city), 
                                     str(state))
    
    # api url
    url = "https://realty-mole-property-api.p.rapidapi.com/rentalPrice"

    # payload
    querystring = {"compCount":str(compCount),
                   "address":address,
                   "squareFootage":squareFootage,
                   "bathrooms":bathrooms,
                   "bedrooms":bedrooms,
                   "propertyType":propertyType,
                   "daysOld":daysOld}

    # header
    headers = {
        'x-rapidapi-key': rapid_api_key,
        'x-rapidapi-host': "realty-mole-property-api.p.rapidapi.com"
        }

    # response
    response = requests.request("GET", url, headers=headers, params=querystring)
    
    # check if there is an error on the request
    if response.status_code == 200: # no error
        return response
    else:
        return """Error {0}. See API source documentation for message and error code - https://rapidapi.com/realtymole/api/realty-mole-property-api/endpoints"""\
        .format(response.status_code)

## <font color="blue">Local & Constants</font>

In [ ]:
# mount drive
drive.mount('/content/drive', force_remount=False)

# data location
file_dir = '/content/drive/My Drive/Colab Data/input/'

Mounted at /content/drive


## <font color="blue">Data</font>

### Get API Keys

In [ ]:
# read in api key file
df_api_keys = pd.read_csv(file_dir + 'api_keys.csv')

# get keys
rapid_api_key = df_api_keys.loc[df_api_keys['API'] =='rapid']['KEY'].iloc[0] # replace this with your own key
google_api_key = df_api_keys.loc[df_api_keys['API'] =='google']['KEY'].iloc[0] # replace this with your own key

## <font color="blue">Test Address</font>
[122 George St, South Amboy, NJ 08879](https://www.zillow.com/homedetails/122-George-St-South-Amboy-NJ-08879/39166231_zpid/)

In [ ]:
# street = "728 June Lake Ln"
# city = "Brandon"
# state = "FL"
# zip_code = "33510"

street = "113 George St"
city = "South Amboy"
state = "NJ"
zip_code = "08879"

## <font color="green">Realty Mole Comparables</font>
<font color="gray">*Search Realty Mole comparables*</font>

[Rapid API - Realty Mole Endpoints](https://rapidapi.com/realtymole/api/realty-mole-property-api/endpoints)

[Realty Mole Website](https://www.realtymole.com/home)

In [ ]:
# get api response
realty_mole_rent_response = realty_mole_rent_estimate(rapid_api_key=rapid_api_key,
                                                    street=street,
                                                    city=city,
                                                    state=state,
                                                    propertyType="Single Family",
                                                    bathrooms=2,
                                                    bedrooms=3,
                                                    daysOld=360,
                                                    compCount=5)

In [ ]:
realty_mole_rent_response.json()

{'latitude': 40.482327,
 'listings': [{'address': '335 Catherine St',
   'bathrooms': 2,
   'bedrooms': 3,
   'city': 'South Amboy',
   'correlation': 0.9809,
   'county': 'Middlesex County',
   'daysOld': 29.91,
   'distance': 0.7130049097964011,
   'formattedAddress': '335 Catherine St, South Amboy, NJ 08879',
   'id': '335-Catherine-St,-South-Amboy,-NJ-08879',
   'latitude': 40.477833,
   'longitude': -74.281954,
   'price': 1200,
   'propertyType': 'Single Family',
   'publishedDate': '2022-01-20T01:14:18.751Z',
   'squareFootage': 1434,
   'state': 'NJ',
   'zipcode': '08879'},
  {'address': '119 John St',
   'bathrooms': 2.5,
   'bedrooms': 3,
   'city': 'South Amboy',
   'correlation': 0.9671,
   'county': 'Middlesex County',
   'daysOld': 113.85,
   'distance': 0.07655265669433753,
   'formattedAddress': '119 John St, South Amboy, NJ 08879',
   'id': '119-John-St,-South-Amboy,-NJ-08879',
   'latitude': 40.482781,
   'longitude': -74.276633,
   'price': 2900,
   'propertyType': 

In [ ]:
# transform to dataframe
df_realty_rental_comps = pd.DataFrame(realty_mole_rent_response.json()['listings'])
# view contents
df_realty_rental_comps

,id,formattedAddress,longitude,latitude,city,state,zipcode,price,publishedDate,distance,daysOld,correlation,address,county,bedrooms,bathrooms,propertyType,squareFootage
0,"335-Catherine-St,-South-Amboy,-NJ-08879","335 Catherine St, South Amboy, NJ 08879",-74.281954,40.477833,South Amboy,NJ,08879,1200,2022-01-20T01:14:18.751Z,0.713005,29.91,0.9809,335 Catherine St,Middlesex County,3,2.0,Single Family,1434.0
1,"119-John-St,-South-Amboy,-NJ-08879","119 John St, South Amboy, NJ 08879",-74.276633,40.482781,South Amboy,NJ,08879,2900,2021-10-28T02:44:29.490Z,0.076553,113.85,0.9671,119 John St,Middlesex County,3,2.5,Single Family,1592.0
2,"218-George-St,-South-Amboy,-NJ-08879","218 George St, South Amboy, NJ 08879",-74.279553,40.481155,South Amboy,NJ,08879,2050,2021-08-19T01:15:46.796Z,0.331491,183.91,0.9600,218 George St,Middlesex County,3,1.5,Single Family,1520.0
3,"253-9th-St,-Apt-A,-South-Amboy,-NJ-08879","253 9th St, Apt A, South Amboy, NJ 08879",-74.276128,40.465003,South Amboy,NJ,08879,2100,2021-05-21T01:06:52.060Z,1.928533,273.92,0.9473,"253 9th St, Apt A",Middlesex County,3,2.0,Single Family,NaN
4,"25-Adam-Blvd,-South-Amboy,-NJ-08879","25 Adam Blvd, South Amboy, NJ 08879",-74.291604,40.463769,South Amboy,NJ,08879,2850,2021-05-05T00:41:32.522Z,2.454406,289.94,0.9332,25 Adam Blvd,Middlesex County,3,2.0,Single Family,NaN


In [ ]:
# zip coordinates: lattitude and longitude, to plot
zip_lat_lon = list(zip(df_realty_rental_comps['latitude'], df_realty_rental_comps['longitude']))
zip_lat_lon

[(40.477833, -74.281954),
 (40.482781, -74.276633),
 (40.481155, -74.279553),
 (40.4650031, -74.2761278),
 (40.463769, -74.291604)]

In [ ]:
# your Google API key
gmaps.configure(api_key=google_api_key) 
# set figure
fig = gmaps.figure()
# add markers as layer
fig.add_layer(gmaps.marker_layer(zip_lat_lon))
# embed in html to view within google collab
embed_minimal_html('export.html', views=[fig])
IPython.display.HTML(filename="export.html")

# End Notebook